In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import unittest

----
----
### 1. Identification

- Choices: 
    - Enhanced recursion (identification of recursive problems are indicators)
    - Recursion: always be given a **choice** (e.g., to include element or not)
    - Overlap: if **overlapping** sub-problems (multiple recursive calls), then DP!
    
- Optimality:
    - Minimum
    - Maximum
    - Largest
    
    
### 2. Writing the Solution

- First, write **recursive** solution
- Next, **memoize** the solution
- Finally, if required, construct the **bottom-up** solution
    - For many tricky problems, we might require a **bottom-up** solution nevertheless!
    

### 3. Parent DP problems

- **0-1 knapsack**
- **Unbounded knapsack**
- Fibonacci
- **LCS**
- LIS
- Kadane's Algorithm
- Matrix Chain Multiplication
- DP on Trees
- DP on Grid 
- Others
----
----

### [Parent Problem-1] 0-1 knapsack:

- Given **N** items where each item has some **weight** and **profit** associated with it and also given a bag with capacity **W**, [i.e., the bag can hold at most W weight in it]. The task is to put the items into the bag such that the **sum of profits** associated with them is the **maximum** possible. 

- Note: The constraint here is we can either put an item completely into the bag or cannot put it at all [It is not possible to put a part of an item into the bag].

In [3]:
# recursive 
def zero_one_knapsack_recursive(weight, value, tw, n, memo=None): 
    
    if memo is None: 
        memo = {}
    
    # 1. first thing to think out loud: WHAT DOES THIS FUNCTION EVEN RETURN?
        # **maximum** sum of profits!
        
    # BASE CONDITION: think of the smallest valid input 
    if n==0 or tw==0:
        return 0

    if (n, tw) in memo:
        return memo[(n, tw)]
                     
    # 2. Reason about the CHOICE DIAGRAM!
    if weight[n-1] > tw:
        # we CANNOT choose this item
        memo[(n, tw)] = zero_one_knapsack_recursive(weight, value, tw, n-1, memo)
    else:
        # we MAY or MAY not choose
        include = value[n-1] + zero_one_knapsack_recursive(weight, value, tw - weight[n-1], n-1, memo)
        exclude = zero_one_knapsack_recursive(weight, value, tw, n-1, memo)
        
        memo[(n, tw)] = max(include, exclude)
        
    return memo[(n, tw)]


# bottom-up solution
    # step-1: initilization
    # step-2: change recursive calls to iterative code!
    
def zero_one_knapsack_table(weight, value, tw, n, matrix=None):
    
    if matrix is None:
        matrix = [[None for _ in range(tw+1)] for _ in range(n+1)]
        
    # BASE CONDITION changes to INITILIZATION
    for j in range(tw + 1):
        matrix[0][j] = 0   # profit
        
    for i in range(n + 1):
        matrix[i][0] = 0   # profit
    
    # CHOICE DIAGARM changes to Matrix filling
    for i in range(1, n+1):
        for j in range(1, tw+1):
            if weight[i-1] > j: 
                # cannot take item!
                matrix[i][j] = matrix[i-1][j]
            else:
                # choice 
                matrix[i][j] = max( value[i-1] + matrix[i-1][j - weight[i-1]], matrix[i-1][j] )
            
    return matrix[n][tw]

In [4]:
class TestKnapsack_Recursive(unittest.TestCase):
    def test_case_1(self):
        self.assertEqual(zero_one_knapsack_recursive([4, 5, 1], [1, 2, 3], 4, 3), 3)

    def test_case_2(self):
        self.assertEqual(zero_one_knapsack_recursive([2, 3, 4], [3, 4, 5], 5, 3), 7)

    def test_case_3(self):
        self.assertEqual(zero_one_knapsack_recursive([1, 2, 3], [10, 20, 30], 5, 3), 50)

    def test_case_4(self):
        self.assertEqual(zero_one_knapsack_recursive([1, 2, 3], [10, 20, 30], 0, 3), 0)

    def test_case_5(self):
        self.assertEqual(zero_one_knapsack_recursive([5, 10, 15], [10, 20, 30], 20, 3), 40)

# Run only the new test cases
suite = unittest.TestLoader().loadTestsFromTestCase(TestKnapsack_Recursive)
unittest.TextTestRunner().run(suite)

class TestKnapsack_table(unittest.TestCase):
    def test_case_1(self):
        self.assertEqual(zero_one_knapsack_table([4, 5, 1], [1, 2, 3], 4, 3), 3)

    def test_case_2(self):
        self.assertEqual(zero_one_knapsack_table([2, 3, 4], [3, 4, 5], 5, 3), 7)

    def test_case_3(self):
        self.assertEqual(zero_one_knapsack_table([1, 2, 3], [10, 20, 30], 5, 3), 50)

    def test_case_4(self):
        self.assertEqual(zero_one_knapsack_table([1, 2, 3], [10, 20, 30], 0, 3), 0)

    def test_case_5(self):
        self.assertEqual(zero_one_knapsack_table([5, 10, 15], [10, 20, 30], 20, 3), 40)
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestKnapsack_table)
unittest.TextTestRunner().run(suite)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.002s

OK
.....
----------------------------------------------------------------------
Ran 5 tests in 0.002s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

**PS**: 
We are maximizing a **function** associated with that each quanity in the array!

#### Btw, a common problem while memoization in Python that you need to worry about is that:

- In Python, default argument values are evaluated only once when the function is **defined**, not each time the function is **called**
- The dictionary {} is created once when the function is **defined**
- This same dictionary is reused across all subsequent calls unless explicitly modified.

In [5]:
def buggy_function(x, demo_memo={}):
    demo_memo[x] = x * 2
    print(demo_memo)

buggy_function(1)  # Output: {1: 2}
buggy_function(2)  # Output: {1: 2, 2: 4} (Unexpected behavior!)
buggy_function(3)  # Output: {1: 2, 2: 4, 3: 6} (Still carrying previous values)

{1: 2}
{1: 2, 2: 4}
{1: 2, 2: 4, 3: 6}


#### [SubProblem-1] Subset Sum:

Given an **array arr[]** of integers and a **value sum**, the task is to check **if** there is a subset of the given array whose sum is equal to the given sum.

In [6]:
def subsetsum_recursive(array, target, n):
    
    if target==0:
        return True
    
    if n==0:
        return False
        
    include = subsetsum_recursive(array, target-array[n-1], n-1)
    exclude = subsetsum_recursive(array, target, n-1)
    return include or exclude

In [7]:
class SubsetSum_Recursive(unittest.TestCase):
    def test_case_1(self):
        self.assertEqual(subsetsum_recursive([3, 34, 4, 12, 5, 2], 9, 6), True)

    def test_case_2(self):
        self.assertEqual(subsetsum_recursive([3, 34, 4, 12, 5, 2], 30, 6), False)
        
    def test_case_3(self):
        self.assertEqual(subsetsum_recursive([-2, 2, 4, -4, 30, 6], 0, 6), True)
        
    def test_case_4(self):
        self.assertEqual(subsetsum_recursive([-3, 1, 2], -2, 3), True)

# Run only the new test cases
suite = unittest.TestLoader().loadTestsFromTestCase(SubsetSum_Recursive)
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.002s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

**PS**: 
We are NOT maximizing/minimizing a **function** associated with each quantity but instead checking if choosing or not choosing the quantity satistifies some property.

#### [SubProblem-2] Equal Sum Partition:

Given an array **arr[]**, the task is to check **if** it can be partitioned into two parts such that the sum of elements in both parts is the same.

**PS**: 
We are NOT maximizing/minimizing a **function** associated with each quantity but instead checking if choosing or not choosing the quantity satistifies some property.

#### [SubProblem-3] Count of Subsets Sum with a Given Sum:

Given an array **arr[]** of length **n** and an integer **target**, the task is to find the **number of subsets** with a sum equal to target.

In [8]:
def countSubsetSum_recursive(array, target, n):

    if target==0:
        return 1

    if n==0:
        return 0

    include = countSubsetSum_recursive(array, target - array[n-1], n-1)
    exclude = countSubsetSum_recursive(array, target, n-1)
    
    return include + exclude

In [9]:
class countSubsetSum_Recursive(unittest.TestCase):
    def test_case_1(self):
        self.assertEqual(countSubsetSum_recursive([1, 2, 3, 3], 6, 4), 3)

    def test_case_2(self):
        self.assertEqual(countSubsetSum_recursive([1, 1, 1, 1], 1, 4), 4)
        
    def test_case_3(self):
        self.assertEqual(countSubsetSum_recursive([2, 3, 5, 6, 8, 10], 10, 6), 3)
        
# Run only the new test cases
suite = unittest.TestLoader().loadTestsFromTestCase(countSubsetSum_Recursive)
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

**PS**: We are **NOT** maximizing/minimizing a **function** associated with each quantity but instead checking if choosing or not choosing the quantity satistifies some property AND as a consequence of that, we end up reporting the number of subsets that statisfy that property!

### [SubProblem-4] Print All Subsets with a Given Sum (Very Important to get a feel of this!!):

Given an array **arr[]** of length n and an integer **target**, the task is to print all those subsets with a sum equal to target.

In [10]:
def howSubsetSum_recursive(array, target, n):

    if target==0:
        return [[]] # empty subset

    if n==0:
        return []   # no subset possible
    
    include = howSubsetSum_recursive(array, target - array[n-1], n-1)
    
    for each_list in include:
        each_list.append(array[n-1])
        
    exclude = howSubsetSum_recursive(array, target, n-1)
    
    return include + exclude

print( howSubsetSum_recursive([1, 2, 3, 3], 6, 4) )
print( howSubsetSum_recursive([1, 1, 1, 1], 1, 4) )
print( howSubsetSum_recursive([2, 3, 5, 6, 8, 10], 10, 6) )

[[3, 3], [1, 2, 3], [1, 2, 3]]
[[1], [1], [1], [1]]
[[10], [2, 8], [2, 3, 5]]


**PS**: We are **NOT** maximizing/minimizing a **function** associated with each quantity but instead checking if choosing or not choosing the quantity satistifies some property AND as a consequence of that, we end up returninig the list of subsets that statisfy the property!

#### [SubProblem-5] Minimum Subset Sum Difference:

- Given an array **arr[]** of size n, the task is to divide it into **two sets S1** and **S2** such that the absolute difference between their sums is **minimum**. 
- If there is a set S with n elements, then if we assume Subset1 has m elements, Subset2 must have n-m elements and the value of abs(sum(Subset1) – sum(Subset2)) should be minimum.

#### [SubProblem-6] Count of subsets given difference:

- check

#### [SubProblem-7] You are given an integer array nums and an integer target.

- You want to build an **expression** out of nums by adding one of the symbols '+' and '-' before each integer in nums and then concatenate all the integers.

- For example, if nums = [2, 1], you can add a '+' before 2 and a '-' before 1 and concatenate them to build the expression "+2-1".

- Return the number of different expressions that you can build, which evaluates to target. 

----
----
### [Parent Problem-2] Unbounded Knapsack:

- Given a knapsack **weight**, say capacity and a set of **n** items with certain value **vali** and weight **wti**, The task is to fill the knapsack in such a way that we can get the **maximum** profit. 
- This is **different** from the classical Knapsack problem, here we are allowed to use an **unlimited** number of instances of an item.

In [11]:
def zero_one_unbounded_knapsack_recursive(weight, value, tw, n):
    
    if tw==0: # no space
        return 0
    if n==0:
        return 0
        
    # base case
    if weight[n-1] > tw:
        return zero_one_unbounded_knapsack_recursive(weight, value, tw, n-1)
    else:
        # choice
        maxValueInclude = value[n-1] + zero_one_unbounded_knapsack_recursive(weight, value, tw-weight[n-1], n)
        maxValueExclude = zero_one_unbounded_knapsack_recursive(weight, value, tw, n-1)
        
        return max(maxValueInclude, maxValueExclude)

In [12]:
class TestUnboundedKnapsack_Recursive(unittest.TestCase):
    def test_case_1(self):
        weight = [1, 50]
        value  = [1, 30]
        tw     = 100
        self.assertEqual( zero_one_unbounded_knapsack_recursive( weight, value, tw, len(value) ), 100)

    def test_case_2(self):
        weight = [1, 3, 4, 5]
        value  = [10, 40, 50, 70]
        tw     = 8
        self.assertEqual( zero_one_unbounded_knapsack_recursive( weight, value, tw, len(value) ), 110)
        
    def test_case_3(self):
        weight = [1, 1]
        value  = [2, 1]
        tw     = 3
        self.assertEqual( zero_one_unbounded_knapsack_recursive( weight, value, tw, len(value) ), 6)
            
# Run only the new test cases
suite = unittest.TestLoader().loadTestsFromTestCase( TestUnboundedKnapsack_Recursive )
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

#### [SubProblem-1] Rod Cutting:

- Given a rod of length **n** inches and an array **price[]**. 
- price[i] denotes the value of a piece of length i. The task is to determine the **maximum** value obtainable by cutting up the rod and selling the pieces.

- Note: price[] is 1-indexed array.

In [13]:
[ i for i in range(1, 8 + 1) ][::-1]   

[8, 7, 6, 5, 4, 3, 2, 1]

#### [SubProblem-2] Coin Change I

- Given an integer array of **coins[]** of size **n** representing different types of denominations and an **integer sum**, the task is to **count all combinations** of coins to **make a given value sum**.

- Note: Assume that you have an infinite supply of each type of coin.

#### [SubProblem-3] Coin Change II

- Given an array of **coins[]** of size **n** and a target value **sum**, where coins[i] represent the coins of different denominations. You have an infinite supply of each of the coins. 
- The task is to find the **minimum** number of coins required to make the given value sum. If it’s not possible to make a change, return -1.

----
----
### [Parent Problem-3] Longest Common Subsequence (LCS):

- Given two strings, **s1** and **s2**, the task is to find the length of the Longest Common Subsequence. If there is no common subsequence, return 0.

- A subsequence is a string generated from the original string by deleting 0 or more characters and without changing the relative order of the remaining characters. 

- For example , subsequences of “ABC” are **“”, “A”, “B”, “C”, “AB”, “AC”, “BC” and “ABC”**.
- In general a string of length n has 2n subsequences.

In [14]:
def longest_common_subsequence(string1, string2, n, m):
            
    if n==0 or m==0:
        return 0    # length of the empty subsequence
    
    if string1[n-1] == string2[m-1]:
        return 1 + longest_common_subsequence(string1, string2, n-1, m-1)
    else:
        longestLeft  = longest_common_subsequence(string1, string2, n-1,   m)
        longestRight = longest_common_subsequence(string1, string2,   n, m-1)
        return max(longestLeft, longestRight)

# memoized version
def longest_common_subsequence_memo(string1, string2, n, m, memo=None):
    
    if memo is None:
        memo = {}
        
    if (n, m) in memo:
        return memo[(n, m)]
        
    if n==0 or m==0:
        return 0    # length of the empty subsequence
    
    if string1[n-1] == string2[m-1]:
        memo[(n, m)] = 1 + longest_common_subsequence_memo(string1, string2, n-1, m-1, memo)
    else:
        longestLeft  = longest_common_subsequence_memo(string1, string2, n-1,   m, memo)
        longestRight = longest_common_subsequence_memo(string1, string2,   n, m-1, memo)
        memo[(n, m)] = max(longestLeft, longestRight)
    
    return memo[(n, m)]

In [15]:
class TestLCS_Recursive_memo(unittest.TestCase):
    def test_case_1(self):
        string1 = "ABC"
        string2 = "ACD"
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence_memo(string1, string2, n, m), 2)

    def test_case_2(self):
        string1 = "AGGTAB"
        string2 = "GXTXAYB"
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence_memo(string1, string2, n, m), 4)
        
    def test_case_3(self):
        string1 = "ABC"
        string2 = "CBA"
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence_memo(string1, string2, n, m), 1)
        
    def test_case_4(self):
        string1 = "abacada"
        string2 = "a"*500
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence_memo(string1, string2, n, m), 4)
        
# Run only the new test cases
suite = unittest.TestLoader().loadTestsFromTestCase( TestLCS_Recursive_memo )
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [16]:
class TestLCS_Recursive(unittest.TestCase):
    def test_case_1(self):
        string1 = "ABC"
        string2 = "ACD"
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence(string1, string2, n, m), 2)

    def test_case_2(self):
        string1 = "AGGTAB"
        string2 = "GXTXAYB"
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence(string1, string2, n, m), 4)
        
    def test_case_3(self):
        string1 = "ABC"
        string2 = "CBA"
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence(string1, string2, n, m), 1)
        
    def test_case_4(self):
        string1 = "abacada"
        string2 = "a"*500
        n, m = len(string1), len(string2)
        self.assertEqual( longest_common_subsequence(string1, string2, n, m), 4)
        
# Run only the new test cases
suite = unittest.TestLoader().loadTestsFromTestCase( TestLCS_Recursive )
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 8.751s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

#### [SubProblem-1]: Print Longest Common Subseqeunce (LCS)

- Given two strings ‘s1‘ and ‘s2‘, print the longest common sub-sequence.

In [17]:
def print_lcs(string1, string2, n, m):
    
    if n==0 or m==0:
        return 0, "" # length of LCS, and the LCS itself
    
    if string1[n-1]==string2[m-1]:
        curr_len, curr_seq = print_lcs(string1, string2, n-1, m-1)
        return 1 + curr_len, curr_seq + string1[n-1]
    
    else:
        left_len, left_seq = print_lcs(string1, string2, n-1, m)
        right_len, right_seq = print_lcs(string1, string2, n, m-1)
        
        if left_len > right_len:
            return left_len, left_seq
        else:
            return right_len, right_seq

In [18]:
string1 = "ABC"
string2 = "ACD"
n, m = len(string1), len(string2)
print( print_lcs(string1, string2, n, m) )

string1 = "AGGTAB"
string2 = "GXTXAYB"
n, m = len(string1), len(string2)
print( print_lcs(string1, string2, n, m) )

string1 = "ABC"
string2 = "CBA"
n, m = len(string1), len(string2)
print( print_lcs(string1, string2, n, m) )

(2, 'AC')
(4, 'GTAB')
(1, 'C')


#### [SubProblem-2] Print All Common Subseqeunces:

- Print all common subsequences between two strings: string1, string2

- If you can **reason** through this problem and **visulize** all the recrusive calls, then you can solve pretty much any variation (there are **SO MANY** variations of this problem btw) of this problem.

In [19]:
def print_common_subsequences(string1, string2, n, m):
    
    if n==0 or m==0:
        return [""]    # length of the empty subsequence
    
    if string1[n-1] == string2[m-1]:
        common_subs = print_common_subsequences(string1, string2, n-1, m-1)       
        new_common_subs = [sub + string1[n-1] for sub in common_subs] # make sure to create a new list       
        return new_common_subs

    else:
        common_subs_left  = print_common_subsequences(string1, string2, n-1,   m)
        common_subs_right = print_common_subsequences(string1, string2,   n, m-1)
        return list( set(common_subs_left + common_subs_right) )
    
    
def print_common_subsequences_memo(string1, string2, n, m, memo=None):
    
    if memo is None:
        memo={}
    
    if n==0 or m==0:
        return [""]    # length of the empty subsequence
    
    if (n, m) in memo:
        return memo[(n, m)]
    
    if string1[n-1] == string2[m-1]:
        common_subs = print_common_subsequences_memo(string1, string2, n-1, m-1, memo)       
        new_common_subs = [sub + string1[n-1] for sub in common_subs] # make sure to create a new list
        memo[(n, m)] = new_common_subs

    else:
        common_subs_left  = print_common_subsequences_memo(string1, string2, n-1,   m, memo)
        common_subs_right = print_common_subsequences_memo(string1, string2,   n, m-1, memo)
        memo[(n, m)] = list( set(common_subs_left + common_subs_right) )    
    
    return memo[(n, m)]

In [20]:
import time

string1 = "abacada" 
string2 = "a"*400
n, m = len(string1), len(string2)

start_time = time.time()
print( print_common_subsequences(string1, string2, n, m) )
end_time = time.time()

print(f"Total time taken is: {end_time - start_time}")

['a', 'aaa', 'aaaa', 'aa']
Total time taken is: 8.20619010925293


In [21]:
string1 = "abacada"
string2 = "a"*400
n, m = len(string1), len(string2)

start_time = time.time()
print( print_common_subsequences_memo(string1, string2, n, m) )
end_time = time.time()

print(f"Total time taken is: {end_time - start_time}")

['a', 'aaa', 'aaaa', 'aa']
Total time taken is: 0.0015990734100341797


#### [SubProblem-3]: Longest Common Substring

- Given two strings ‘s1‘ and ‘s2‘, find the **length** of the longest common **substring**.

#### [SubProblem-4]: Print Longest Common Substring

- Given two strings ‘s1‘ and ‘s2‘, print the longest common **substring**.

#### [SubProblem-5]: Shortest Common Super Sequence

- Given two strings ‘s1‘ and ‘s2‘, find the **length** of the shortest string that has both s1 and s2 as subsequences.

#### [SubProblem-6]: Print Shortest Common Super Sequence

- Given two strings s1 and s2, print the **shortest string** which has both s1 and s2 as its sub-sequences. If multiple shortest super-sequence exists, print any one of them.

In [22]:
def print_scs(string1, string2, n, m):
    
    # base condition
    if n==0:
        return m, string2[:m] # key!
    if m==0:
        return n, string1[:n]
    
    # choice diagram!
    if string1[n-1]==string2[m-1]:
        curr_len, curr_seq = print_scs(string1, string2, n-1, m-1)
        return 1 + curr_len, curr_seq + string1[n-1]
    
    else:
        left_len, left_seq = print_scs(string1, string2, n-1, m)
        right_len, right_seq = print_scs(string1, string2, n, m-1)
        
        if left_len < right_len:
            return 1 + left_len, left_seq + string1[n-1]
        else:
            return 1 + right_len, right_seq + string2[m-1]

In [23]:
string1 = "ABC"
string2 = "ACD"
n, m = len(string1), len(string2)
print( print_scs(string1, string2, n, m) )

string1 = "AGGTAB"
string2 = "GXTXAYB"
n, m = len(string1), len(string2)
print( print_scs(string1, string2, n, m) )

string1 = "ABC"
string2 = "CBA"
n, m = len(string1), len(string2)
print( print_scs(string1, string2, n, m) )

(4, 'ABCD')
(9, 'AGGXTXAYB')
(5, 'ABCBA')


#### [SubProblem-7]: Minimum Number of Insertion and Deletion to convert String A to String B

- Given two strings s1 and s2. The task is to remove/delete and insert the minimum number of characters from s1 to transform it into s2. 

- It could be possible that the same character needs to be removed/deleted from one point of s1 and inserted at another point.

#### [SubProblem-8]: Longest Palindromic Subsequence (LPS)

- Given a string **s**, find the length of the Longest Palindromic Subsequence in it.

- Note: The Longest Palindromic Subsequence (LPS) is the maximum-length subsequence of a given string that is also a Palindrome.

#### [SubProblem-9]:  Minimum number of deletion in a string to make it a palindrome

- Given a string of size ‘n’. The task is to remove or delete the minimum number of characters from the string so that the resultant string is a palindrome. 

- Note: The order of characters should be maintained.

#### [SubProblem-10]: Longest repeating subsequence

- Given a string s, the task is to find the longest repeating subsequence, such that the two subsequences don’t have the same string character at the same position, i.e. any $i^{th}$ character in the two subsequences shouldn’t have the same index in the original string.

#### [SubProblem-11]: Sequence Pattern Matching

- Given two strings s and t, return true if s is a subsequence of t, or false otherwise.

- A subsequence of a string is a new string that is formed from the original string by deleting some (can be none) of the characters without disturbing the relative positions of the remaining characters.

- (i.e., "ace" is a subsequence of "abcde" while "aec" is not).

#### [SubProblem-12]: Minimum insertions to form a palindrome

- Given string str, the task is to find the minimum number of characters to be inserted to convert it to a palindrome.

----
----
### [Parent Problem-4]: Find number of times string A occurs as a subsequence string B

- Given two strings, find the number of times the second string occurs in the first string, whether continuous or discontinuous.

#### [SubProblem-1]: Distinct Subsequences (With Repetition)

- Problem: Count the number of distinct subsequences of A.

In [28]:
def subseqeunces(string, n):
    
    if n==0:
        return [""]
    
    current_sub_seq = subseqeunces(string, n-1)
    
    new_include = [sub_seq + string[n-1] for sub_seq in current_sub_seq]
    
    return current_sub_seq + new_include

string = "mm"
n = len(string)
values = subseqeunces(string, n)
print(len(values), values)

4 ['', 'm', 'm', 'mm']


#### [SubProblem-2]: Count Subsequences That Follow a Pattern

- Given a pattern (like "abc"), count how many times it appears in B as a subsequence.

#### [SubProblem-3]: Lexicographically Smallest Subsequence

- Find the smallest lexicographic subsequence of B that contains all characters of A.

----
----
### [Parent Problem-5]: Matrix Chain Multiplication

- 